In [10]:
import sys
import DualNum
import csv
import numpy as np
import tSNE
import multiprocessing
import datetime
import time
import Grid
import json
import ProjectionRunner
from ProjectionRunner import ProjectionRunner

## Supporting Functions

In [3]:
# read points of cvs file
def readFile(filename):
    read = csv.reader(open(filename, 'rt'))

    points = []
    firstLine = next(read)
    headers = []
    rowDat = []
    head = False
    for i in range(0, len(firstLine)):
        try:
            rowDat.append(float(firstLine[i]))
        except:
            head = True
            break
    if head:
        headers = firstLine
    else:
        points.append(rowDat)

    for row in read:
        rowDat = []
        for i in range(0, len(row)):
            try:
                rowDat.append(float(row[i]))
            except:
                print("invalid data type - must be numeric")
                exit(0)
        points.append(rowDat)
    return points

In [21]:
def runProjection(projection, points, perturbations, parameters,filePrefix):
    pr = ProjectionRunner(projection,parameters)

    n = len(points)

    pts = []
    print("Running DimReader")
    for i in range(len(perturbations)):
        pert = perturbations[i]
        pr.calculateValues(np.array(points), np.array(pert))

        derivVect = pr.resultVect
        print(derivVect)
        projPts = pr.points
        data = []
        for j in range(n):
            data.append({"domain":points[j],
                         "range": projPts[j],
                         "inputPert": pert[j],
                         "outputPert":[derivVect[2*j],derivVect[2*j+1]]
                        })
        if len(perturbations)>1:
            fileName = filePrefix +"_"+str(i)+".dimreader"
        else:
            fileName = filePrefix + ".dimreader"


        grid = calcGrid(projPts,derivVect)
        output = {"points": data,
                  "scalarField": grid}
        f = open(fileName, "w")
        f.write(json.dumps(output))
        f.close()


In [5]:
def calcGrid(points,dVects):#date, grid, gridCoord, ind):
    gridCoord = generateGrid(points)
    print(gridCoord)
    g = Grid.Grid(points, dVects, gridCoord)

    grid = g.calcGridPoints()

    n = len(gridCoord)
    m = len(gridCoord[0])
    gridPoints = []
    for i in range(n):
        gridPoints.append([])
        for j in range(m):
            gridPoints[i].append(grid[m * i + j])


    return grid.tolist()

In [6]:
def generateGrid(points):
    gridSize = 10
    points = np.array(points)
    xmax = max(points[:, 0])
    xmin = min(points[:, 0])
    ymax = max(points[:, 1])
    ymin = min(points[:, 1])

    gridCoord = []
    if (xmin == xmax):
        xmax += 1
        xmin -= 1
    if (ymin == ymax):
        ymin -= 1
        ymax += 1
    # if ymax > xmax:
    #     xmax = ymax
    # else:
    #     ymax = xmax
    # if ymin < xmin:
    #     xmin = ymin
    # else:
    #     ymin = xmin
    yrange = ymax - ymin
    xrange = xmax - xmin

    xstep = float(xrange) / (gridSize - 1)
    ystep = float(yrange) / (gridSize - 1)

    for i in range(gridSize + 1):
        gridCoord.append([])
        for j in range(gridSize + 1):
            gridCoord[i].append([(xmin - xstep / 2.0) + xstep * j, (ymax + ystep / 2.0) - ystep * i])
    return gridCoord

## Scripts

In [16]:
projections = ["tsne", "Tangent-Map"]
projectionClasses=[tSNE.tSNE,None]
projectionParamOpts = [tSNE.paramOpts,[]]

inputFile = "irisMini.csv"
perturbFile = "all"
projection = "tsne"
params = [30]

In [20]:
projInd = list(map(str.lower, projections)).index(str.lower(projection))
# print(projInd)

date = str(datetime.datetime.fromtimestamp(time.time())).replace(" ", "_")
date = date.split(".")[0]

# prefix of output file
prefix = inputFile[:inputFile.rfind(".")] + "_" + projections[projInd] + "_" + date
# print(prefix)

if str.lower(projection) != "tangent-map":
    inputPts = readFile(inputFile)

    if str.lower(perturbFile) == "all":
        perturbVects = []
        n, m = np.shape(inputPts)
        for i in range(m):
            currPert = np.zeros((n, m))
            for j in range(n):
                currPert[j][i] = 1
            perturbVects.append(currPert.tolist())
    else:
        perturbVects = [readFile(perturbFile)]
        
print("Input Points:\n",inputPts)
print("Pertabation Vectors:\n",perturbVects)

Input Points:
 [[5.1, 3.5, 1.4, 0.2], [4.9, 3.0, 1.4, 0.2], [4.7, 3.2, 1.3, 0.2], [4.6, 3.1, 1.5, 0.2], [5.0, 3.6, 1.4, 0.2]]
Pertabation Vectors:
 [[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0]], [[0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]], [[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0]], [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0]]]


In [19]:
runProjection(projectionClasses[projInd], inputPts, perturbVects,params,prefix)
print("Output File Prefix: ",prefix)

Running DimReader
Min:  0 I:  0 max:  5
Min:  0 I:  1 max:  5
Min:  0 I:  2 max:  5
Min:  0 I:  3 max:  5
Min:  0 I:  4 max:  5
[2.1982913643760437e-32, -2.1688194470297363e-31, 3.6609262192922933e-47, -7.180203391170358e-47, 1.0563871953948953e-61, -5.848923825617627e-62, 8.703753419187568e-77, -7.413074870049393e-77, 6.708122082172877e-92, -9.973113495721861e-93]
[[[-395.58469859824015, 391.0835526168602], [-314.00827375703653, 391.0835526168602], [-232.43184891583292, 391.0835526168602], [-150.8554240746293, 391.0835526168602], [-69.27899923342568, 391.0835526168602], [12.297425607777939, 391.0835526168602], [93.87385044898156, 391.0835526168602], [175.45027529018523, 391.0835526168602], [257.0267001313888, 391.0835526168602], [338.60312497259235, 391.0835526168602], [420.17954981379603, 391.0835526168602]], [[-395.58469859824015, 306.914588850349], [-314.00827375703653, 306.914588850349], [-232.43184891583292, 306.914588850349], [-150.8554240746293, 306.914588850349], [-69.27899923